<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 理解嵌入层和线性层之间的区别
- PyTorch中的嵌入层实现的功能与执行矩阵乘法的线性层相同；我们使用嵌入层的原因是计算效率
- 将使用PyTorch中的代码示例逐步查看这种关系

In [1]:
import torch

print("PyTorch version:", torch.__version__)

PyTorch version: 2.2.1+cu118


<br>
&nbsp;

## 使用嵌入层，nn.Embedding

In [2]:
# 假设现有以下三个训练样本，表示LLM中的tokens id
idx = torch.tensor([2, 3, 1])

# 嵌入矩阵的行数可以通过获取最大标记ID + 1来确定
# 如果最高标记ID是3，那么我们需要4行，对应可能的标记ID 0、1、2、3
num_idx = max(idx) + 1

# 所需的嵌入维度是一个超参数
out_dim = 5

- 以下实现一个简单的嵌入层

In [4]:
# 使用随机种子以确保可重复性，因为嵌入层重点额权重是用小的随机值初始化的
torch.manual_seed(123)

embedding = torch.nn.Embedding(num_idx, out_dim)

可以选择性看一下嵌入层的权重

In [5]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

- 我们可以通过嵌入层获得一个训练例子的向量表示

In [6]:
embedding(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 以下是具体发生的可视化过程

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/1.png" width="400px">
</p>

- 相同，可以获得其他tokend的嵌入表示

In [7]:
embedding(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/2.png" width="400px">
</p>

- 相同地，将之前定义的训练样本整体转换

In [8]:
idx = torch.tensor([2, 3, 1])
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 在底层，它仍然是相同的查找概念

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/3.png" width="450px">
</p>

## 使用线性层，nn.Linear
- 以下将演示上面的嵌入层与在PyTorch中对独热编码表示使用nn.Linear层实现的效果完全相同
- 首先，先将标记ID转换为独热表示

In [9]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

- 接下来，初始化一个Linear层，其执行矩阵乘法$X W^\top$

In [10]:
torch.manual_seed(123)
linear = torch.nn.Linear(num_idx, out_dim, bias=False)
linear.weight

Parameter containing:
tensor([[-0.2039,  0.0166, -0.2483,  0.1886],
        [-0.4260,  0.3665, -0.3634, -0.3975],
        [-0.3159,  0.2264, -0.1847,  0.1871],
        [-0.4244, -0.3034, -0.1836, -0.0983],
        [-0.3814,  0.3274, -0.1179,  0.1605]], requires_grad=True)

- 请注意，PyTorch中的线性层也是用小的随机权重初始化的；为了直接将其与上面的Embedding层进行比较，必须使用相同的小随机权重，这就是为什么在这里重新分配它们


In [11]:
linear.weight = torch.nn.Parameter(embedding.weight.T)

- 现在可以在输入的独热编码表示上使用线性层

In [12]:
linear(onehot.float())

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]], grad_fn=<MmBackward0>)

- 对比可知，这与使用嵌入层时得到的结果完全相同

In [13]:
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 在底层发生的是对第一个训练样本的标记ID进行以下计算

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/4.png" width="450px">
</p>

- 对于第二个训练样本的标记ID

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/5.png" width="450px">
</p>

- 由于每个独热编码行中除了一个索引外都是0（根据设计），这个矩阵乘法本质上与独热元素的查找相同
- 在独热编码上使用矩阵乘法等同于嵌入层查找，但如果使用大型嵌入矩阵，这可能效率低下，因为有很多与零相乘的无用计算